In [7]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

In [34]:
df = pd.read_csv('./data-test.csv', index_col=False, encoding='utf-8', error_bad_lines=False)
df.head()

b'Skipping line 7947: expected 7 fields, saw 9\nSkipping line 8369: expected 7 fields, saw 8\nSkipping line 11767: expected 7 fields, saw 8\n'


,number,Comment,Sentiment,Sentiment_Mixed,Sentiment_Negative,Sentiment_Neutral,Sentiment_Positive
0,0,"Mira todo el vídeo para saber cómo, cuándo y d...",NEUTRAL,7.38355447538197e-05,0.07488377392292023,0.7302832007408142,0.19475926458835602
1,1,La exbox es el que yo quiero,POSITIVE,0.002404651138931513,0.01960178092122078,0.2622445225715637,0.7157490253448486
2,2,N.o. m.e. i.g.n.o.r.e.n,NEUTRAL,2.4324339392478578e-05,0.14702238142490387,0.8462755084037781,0.0066777500323951244
3,3,Ya me siscribi,NEUTRAL,1.953602804860566e-06,0.31459498405456543,0.6729583740234375,0.012444673106074333
4,4,Quiero participar,NEUTRAL,2.2022537450538948e-05,0.0019117718329653144,0.6043353080749512,0.39373087882995605


In [35]:
df = df[df['Sentiment'].isin(['POSITIVE', 'NEGATIVE'])]
df.count()

number                7546
Comment               7546
Sentiment             7546
Sentiment_Mixed       7546
Sentiment_Negative    7546
Sentiment_Neutral     7546
Sentiment_Positive    7546
dtype: int64

In [72]:
aaa = list(df[df['Sentiment'] == 0]['Comment'])
print(aaa[0])

no puedo estar en todos lo directo por que no tengo celular


In [46]:
df['Comment'] = df['Comment'].apply(lambda x: str(x).lower())
df['Sentiment'] = df['Sentiment'].apply(lambda x: 1 if x == 'POSITIVE' else 0)
df.head()

,number,Comment,Sentiment,Sentiment_Mixed,Sentiment_Negative,Sentiment_Neutral,Sentiment_Positive
1,1,la exbox es el que yo quiero,1,0.002404651138931513,0.01960178092122078,0.2622445225715637,0.7157490253448486
6,6,yo quisiera que me tocara la xbox es mi consol...,1,0.00023217413399834186,0.006433135364204645,0.0954638123512268,0.8978709578514099
7,7,me encantaría la negra,1,0.1305803507566452,0.027237318456172943,0.13413338363170624,0.7080488801002502
8,8,like: listo\nsuscrita a los dos canales:listo ...,1,6.439394837798318e-06,0.0006571895792149007,0.3668906092643738,0.6324456930160522
11,11,buenas me gustaría ganar la ps5 soy muy fan d...,1,7.720212306594476e-05,0.027157854288816452,0.17522844672203064,0.7975365519523621


In [47]:
vectorizer = CountVectorizer(
    analyzer = 'word',
    lowercase = False,
)
features = vectorizer.fit_transform(
    df['Comment']
)
features_nd = features.toarray() # for easy usage


In [48]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test  = train_test_split(
        features_nd, 
        df['Sentiment'],
        train_size=0.80, 
        random_state=1234)

In [55]:
from sklearn.linear_model import LogisticRegression

log_model = LogisticRegression(solver='lbfgs',class_weight='balanced', max_iter=10000)
log_model = log_model.fit(X=X_train, y=y_train)
y_pred = log_model.predict(X_test)
df.head()

,number,Comment,Sentiment,Sentiment_Mixed,Sentiment_Negative,Sentiment_Neutral,Sentiment_Positive
1,1,la exbox es el que yo quiero,1,0.002404651138931513,0.01960178092122078,0.2622445225715637,0.7157490253448486
6,6,yo quisiera que me tocara la xbox es mi consol...,1,0.00023217413399834186,0.006433135364204645,0.0954638123512268,0.8978709578514099
7,7,me encantaría la negra,1,0.1305803507566452,0.027237318456172943,0.13413338363170624,0.7080488801002502
8,8,like: listo\nsuscrita a los dos canales:listo ...,1,6.439394837798318e-06,0.0006571895792149007,0.3668906092643738,0.6324456930160522
11,11,buenas me gustaría ganar la ps5 soy muy fan d...,1,7.720212306594476e-05,0.027157854288816452,0.17522844672203064,0.7975365519523621


In [75]:
import random
j = random.randint(0,len(X_test)-7)
comments = list(df['Comment'])
for i in range(j,j+7):
    print(y_pred[i])
    ind = features_nd.tolist().index(X_test[i].tolist())
    print(comments[i])

1
fanático de la play station 🚀❤️
1
la play 5 me gustaria ganar y por si a caso ya me suscribi y active la campanita
1
a mi no me importa cual de las dos ganar,"si gano"solo quiero jugar con mis amigos
0
no entendi como participar :(
1
me gustaría ganar la ps 5 y necesito jugar el spiderman mails morales de verdad doy fan de spiderman
0
hola, muy buen canal. me gustaría la ps5 ya que arranque con las consolas de sony desde 1997. saludos
1
saludos desde argentina


In [58]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.8556291390728477
